Scraping for twitter

In [27]:
# from twitter_scraper_selenium import scrape_keyword
# #scrap 10 posts by searching keyword "india" from date 30th August till date 31st August
# cancer = scrape_keyword(keyword="#cancer #nonsmoking", browser="chrome",
#                       tweets_count=1,output_format="json" ,until="2023-08-31", since="2021-08-30")
# print(cancer)

Database Connection

In [2]:
import sqlite3

conn = sqlite3.connect('cancer.db')

try:
    conn.cursor()
    print("DB Connected");
except Exception as ex:
    print("DB not Connected");

DB Connected


FOLLOWING TABLES ARE CREATED -
User Table : This table contains the details of the user, who have tweeted
Cancer Type Table : This table holds different categories of cancer
Treatment Table: This table contains types of treatment based on Cancer type
Location Table:This table describes the location of the user
Smoking Table: This table briefs whether user is a smoker or not

In [ ]:
#USER TABLE
conn = sqlite3.connect('cancer.db')
print ("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS USER;''')
conn.execute('''CREATE TABLE IF NOT EXISTS USER
         (Id INTEGER PRIMARY KEY AUTOINCREMENT,
         User_Name     TEXT   NOT NULL,
         Handle_Name   TEXT   NOT NULL,
         Description   TEXT   NOT NULL,
         Followers_Count     INTEGER,
         Created_Date   TEXT  NOT NULL,
         Image_Url      TEXT  NOT NULL);''')

print ("User Table created successfully");

conn.close()

#CANCER_TYPE TABLE
conn = sqlite3.connect('cancer.db')
print ("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS CANCER_TYPE;''')
conn.execute('''CREATE TABLE IF NOT EXISTS CANCER_TYPE
         (Cancer_Id INTEGER PRIMARY KEY AUTOINCREMENT,
         Cancer_Type      TEXT  NOT NULL);''')

print ("Cancer_Type Table created successfully");

conn.close()

#TREATMENT TABLE
conn = sqlite3.connect('cancer.db')
print ("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TREATMENT;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TREATMENT
         (Treatment_Id INTEGER PRIMARY KEY AUTOINCREMENT,
          Treatment_Type      TEXT  NOT NULL);''')

print ("Treatment Table created successfully");

conn.close()

#LOCATION TABLE
conn = sqlite3.connect('cancer.db')
print ("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS LOCATION;''')
conn.execute('''CREATE TABLE IF NOT EXISTS LOCATION
         (Location_Id INTEGER PRIMARY KEY AUTOINCREMENT,
          Location     TEXT  NOT NULL);''')

print ("Location Table created successfully");

conn.close()

#SMOKING TABLE
conn = sqlite3.connect('cancer.db')
print ("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS SMOKING;''')
conn.execute('''CREATE TABLE IF NOT EXISTS SMOKING
         (Cancer_Id INTEGER NOT NULL,
          Smoking_Id INTEGER PRIMARY KEY AUTOINCREMENT,
          Smoker_Or_Not  INTEGER NOT NULL,
          FOREIGN KEY(Cancer_Id) REFERENCES CANCER_TYPE(Cancer_Id) );''')

print ("Smoking Table created successfully");

conn.close()

FOLLOWING TABLES ARE CREATED -
Tweets Table : This table contains the details of the tweet with the tweet handle name and the posted date
Hashtag Table : This table holds details on the hashtag and the tweet_id
Tweet mention Table: This table contains information about the tweet mention, source user and target user
Tweet URL Table: This table describes the URL of the tweet and the URL Id

In [28]:
#TWEETS TABLE
conn = sqlite3.connect('cancer.db')
print ("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TWEETS;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TWEETS
         (Tweet_Id INTEGER PRIMARY KEY NOT NULL,
          Tweet_Handle TEXT NOT NULL,
          Tweet_Text TEXT NOT NULL,
          Posted_At DATE,
          Id INTEGER NOT NULL,
          FOREIGN KEY(Id) REFERENCES USER(Id));''')

print ("Tweets Table created successfully");

conn.close()


#HASHTAG TABLE
conn = sqlite3.connect('cancer.db')
print ("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS HASHTAG;''')
conn.execute('''CREATE TABLE IF NOT EXISTS HASHTAG
         (Hashtag_Id INTEGER PRIMARY KEY NOT NULL,
          Hashtag_Text TEXT NOT NULL,
          Tweet_Id  INTEGER NOT NULL,
          FOREIGN KEY(Tweet_Id) REFERENCES TWEETS(Tweet_Id));''')

print ("Hashtag Table created successfully");

conn.close()


#TWEET MENTION TABLE
conn = sqlite3.connect('cancer.db')
print ("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TWEET_MENTION;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TWEET_MENTION
         (Mention_Id INTEGER PRIMARY KEY NOT NULL,
          Source_User TEXT NOT NULL,
          Target_User TEXT NOT NULL,
          Tweet_Id  INTEGER NOT NULL,
          FOREIGN KEY(Tweet_Id) REFERENCES TWEETS(Tweet_Id));''')

print ("Tweet Mention Table created successfully");

conn.close()


#TWEET URL TABLE
conn = sqlite3.connect('cancer.db')
print ("Opened database successfully");
conn.execute('''DROP TABLE IF EXISTS TWEET_URL;''')
conn.execute('''CREATE TABLE IF NOT EXISTS TWEET_URL
         (Url_Id INTEGER PRIMARY KEY NOT NULL,
          Tweet_Url TEXT NOT NULL,
          Tweet_Id  INTEGER NOT NULL,
          FOREIGN KEY(Tweet_Id) REFERENCES TWEETS(Tweet_Id));''')

print ("Tweet URL Table created successfully");

conn.close()


Opened database successfully
Tweets Table created successfully
Opened database successfully
Hashtag Table created successfully
Opened database successfully
Tweet Mention Table created successfully
Opened database successfully
Tweet URL Table created successfully
